In [1]:
import pandas as pd
import numpy as np
import pyodbc
import math
from ast import literal_eval
from tqdm import tqdm
import os
from configparser import ConfigParser
import statistics
from scipy import stats as s

In [2]:
pd.set_option("display.max_column",200)
pd.set_option("display.float_format",lambda x:'%.5f' %x)

In [3]:
data_ind_U  = pd.read_csv(r"/data/Synthetic_data_Clustered_individual_U.csv")
ind_data_thresholds_U = pd.read_csv(r"/data/threshold_individual_U.csv")
weight_table_ind_U = pd.read_csv(r"/data/weight_individual_U.csv")

In [4]:
ind_data_thresholds_U

O002003003  \
0  (None, None, None, None, None, None, None, Non...   
1  (None, None, None, None, None, None, None, Non...   
2  (None, None, None, None, None, None, None, Non...   
3  (None, None, None, None, None, None, None, Non...   
4  (None, None, None, None, None, None, None, Non...   
5  (None, None, None, None, None, None, None, Non...   
6  (None, None, None, None, None, None, None, Non...   
7  (None, None, None, None, None, None, None, Non...   
8  (None, None, None, None, None, None, None, Non...   

                                          O002001030  \
0  (None, None, None, None, None, None, None, Non...   
1  (None, None, None, None, None, None, None, Non...   
2  (None, None, None, None, None, None, None, Non...   
3  (None, None, None, None, None, None, None, Non...   
4  (None, None, None, None, None, None, None, Non...   
5  (None, None, None, None, None, None, None, Non...   
6  (None, None, None, None, None, None, None, Non...   
7  (None, None, None, None, None, None, None, Non...   
8  (None, None, None, None, None, None, None, Non...   

                                          O002003005  \
0  (None, None, None, None, None, None, None, Non...   
1  (None, None, None, None, None, None, None, Non...   
2  (-179, -178.0, -177.0, -176.0, -175.0, -174.0,...   
3  (None, None, None, None, None, None, None, Non...   
4  (None, None, None, None, None, None, None, Non...   
5  (None, None, None, None, None, None, None, Non...   
6  (None, None, None, None, None, None, None, Non...   
7  (None, None, None, None, None, None, None, Non...   
8  (None, None, None, None, None, None, None, Non...   

                                          O002001004  \
0  (2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 1.47826086956...   
1  (2, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 1...   
2  (2, 4, 5, 7, 8, 9, 11, 12, 14, 15, 2.0, 1.0, 1...   
3  (1, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10...   
4  (2, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 1...   
5  (2, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 1...   
6  (3, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, ...   
7  (2, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 1...   
8  (3, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, ...   

                                          O002001032  \
0  (None, None, None, None, None, None, None, Non...   
1  (None, None, None, None, None, None, None, Non...   
2  (None, None, None, None, None, None, None, Non...   
3  (None, None, None, None, None, None, None, Non...   
4  (None, None, None, None, None, None, None, Non...   
5  (None, None, None, None, None, None, None, Non...   
6  (3, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, ...   
7  (None, None, None, None, None, None, None, Non...   
8  (3, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, ...   

                                          O002003002  \
0  (None, None, None, None, None, None, None, Non...   
1  (None, None, None, None, None, None, None, Non...   
2  (None, None, None, None, None, None, None, Non...   
3  (254255, 254256.0, 254257.0, 254258.0, 254259....   
4  (None, None, None, None, None, None, None, Non...   
5  (None, None, None, None, None, None, None, Non...   
6  (None, None, None, None, None, None, None, Non...   
7  (None, None, None, None, None, None, None, Non...   
8  (None, None, None, None, None, None, None, Non...   

                                          O002001019  \
0  (None, None, None, None, None, None, None, Non...   
1  (None, None, None, None, None, None, None, Non...   
2  (None, None, None, None, None, None, None, Non...   
3  (None, None, None, None, None, None, None, Non...   
4  (None, None, None, None, None, None, None, Non...   
5  (None, None, None, None, None, None, None, Non...   
6  (3, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, ...   
7  (None, None, None, None, None, None, None, Non...   
8  (None, None, None, None, None, None, None, Non...   

                                          O002001021  \
0  (2, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 1...   
1  

In [5]:
ind_data_thresholds_U['PEER_GROUP_ID'] = ind_data_thresholds_U.apply(lambda row:row['OCCUPATION_BUSINESS_CLEAN']+"_"+str(row['Cluster_labels']),axis=1)
weight_table_ind_U['PEER_GROUP_ID'] = weight_table_ind_U.apply(lambda row:row['OCCUPATION_BUSINESS_CLEAN']+"_"+str(row['Cluster_labels']),axis=1)

In [6]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [8]:
# Changed Code

count = 1
# Populating the threshold values in the threshold table
for ind,row in ind_data_thresholds_U.iterrows():
    if (count%10000 == 0):
        print(count," Done")
    count+=1
    for key in row.keys():
        
        if key.startswith('O0'): #only considering columns which start with P
            
            data = literal_eval(row[key])
#             print(row['PEER_GROUP_ID'],key)
            print("data = ",data)

            if all(elem is None for elem in data) == True:
                print("The rule_no",key,"don't have any value for peer_group",row['PEER_GROUP_ID'])
                conn.execute("""insert into fincore_db.fincore_analytics.RISK_LKP_ENTITY_THRESHOLD (PEER_GROUP_ID,RULE_NO)values(?,?)""",(row['PEER_GROUP_ID'],key))
                print("end")

            else:   
#                 print("data = ",data)
# #                 data = literal_eval(row[key])
                conn.execute("""insert into fincore_db.fincore_analytics.RISK_LKP_ENTITY_THRESHOLD (THRESHOLD1_1,THRESHOLD1_2,THRESHOLD1_3,THRESHOLD1_4,THRESHOLD1_5,THRESHOLD1_6,THRESHOLD1_7,THRESHOLD1_8,THRESHOLD1_9,THRESHOLD1_10,MEAN,MEDIAN,MODE,STANDARD_DEVIATION,OUTLIER,PEER_GROUP_ID,RULE_NO)values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""",(data[0],data[1],data[2],data[3],data[4],data[5],data[6],data[7],data[8],data[9],data[10],data[11],data[12],data[13],data[14],row['PEER_GROUP_ID'],key))
conn.commit()
# conn.close()

data =  (8, 13, 19, 24, 30, 35, 41, 46, 52, 57, 5.0, 2.0, 1, 5.529143565818179, 19.5)
data =  (128481.0, 128481.0, 128481.0, 128481.0, 128481.0, 128481.0, 128481.0, 128481.0, 128481.0, 128481.0, 128481.0, 128481.0, 128481, 0.0, 'NA')
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no O002001032 don't have any value for peer_group OTHERS_CHANGED_9
end
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no O002001026 don't have any value for peer_group OTHERS_CHANGED_9
end
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no O002003007 don't have any value for peer_group OTHERS_CHANGED_9
end
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no O002001019 don't have any value for peer_group OTHERS_CHANGED_9
end
data =  (2, 3, 5, 6, 7, 8, 10, 11, 12, 13, 1.6666666666666667, 1.0, 1, 1.247219128924647, 6.5)
data =  (1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1, 0.0

data =  (2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 1.2, 1.0, 1, 0.4000000000000001, 2.5)
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no O002001031 don't have any value for peer_group OTHERS_CHANGED_1
end
data =  (1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1, 0.0, 'NA')
data =  (1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1, 0.0, 'NA')
data =  (-24.0, -24.0, -24.0, -24.0, -24.0, -24.0, -24.0, -24.0, -24.0, -24.0, -24.0, -24.0, -24, 0.0, 'NA')
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no O002001023 don't have any value for peer_group OTHERS_CHANGED_1
end
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no O002003003 don't have any value for peer_group OTHERS_CHANGED_1
end
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no O002001021 don't have any value for peer_group OTHERS_CHANGED_1
end
data =  (2.0, 3.0, 4.0, 5.0, 6

data =  (2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2, 0.0, 'NA')
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no O002001018 don't have any value for peer_group OTHERS_CHANGED_2
end
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no O002001008 don't have any value for peer_group OTHERS_CHANGED_2
end
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no O002001025 don't have any value for peer_group OTHERS_CHANGED_8
end
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no O002003005 don't have any value for peer_group OTHERS_CHANGED_8
end
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no O002001032 don't have any value for peer_group OTHERS_CHANGED_8
end
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no O002001026 don't have any value for peer_group OTHERS_CHANGED_8
end
data =  (None, None, No

In [9]:
peers = pd.read_sql_query("Select distinct peer_group_id from fincore_db.fincore_analytics.risk_ORGANIZATION_summary_stg with (nolock) ",conn)

In [10]:
peers = peers['peer_group_id'].to_list()

In [11]:
rule_no = pd.read_sql_query("select rule_no from fincore_db.fincore_analytics.risk_lkp_rules with (nolock) where is_active_flag = 1 and RISK_SCORE_STATIC_FLAG = 0 and RULE_NO like 'O%' and risk_type <> 'Auto High Rules '",conn)

In [12]:
rule_no = rule_no['rule_no'].to_list()

In [13]:
data_ind_U.drop_duplicates(inplace = True)

In [14]:

for peer_group_id in peers:
    for rules in rule_no: 
#         print(peer_group_id,rules)
        a = data_ind_U[data_ind_U['PEER_GROUP_ID'] == peer_group_id][rules]
        a = a.dropna()
        a = a.astype('int32')
        if (len(a) == 0):
            th = ['Null','Null']
        else:
            th = []
#             print(min(a),max(a),np.mean(a),np.median(a),statistics.mode(a))
#             th[0] = min(a)
#             th[1] = max(a)
#             th[2] = np.mean(a)
#             th[3] = np.median(a)
#             th[4] = statistics.mode(a)
            th.append(min(a))
            th.append(max(a))

        cursor.execute("""update fincore_db.fincore_analytics.RISK_LKP_ENTITY_THRESHOLD set min = {} , max = {} ,update_date = getdate() where peer_group_id ='{}' and rule_no = '{}' """.format(th[0],th[1],peer_group_id,rules))
        conn.commit()
        
conn.close()



In [1]:
print('Entity_Risk_Calculation_V2_1_org completed.')

Entity_Risk_Calculation_V2_1_org completed.
